In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""
CELL 1: Setup and Imports
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Create output directory for Kaggle
os.makedirs('/kaggle/working/outputs', exist_ok=True)
OUTPUT_DIR = '/kaggle/working/outputs/'

# Set plot style for better visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check TensorFlow version and GPU availability
print("=" * 70)
print("ENVIRONMENT SETUP")
print("=" * 70)
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Output directory: {OUTPUT_DIR}")
print("=" * 70)
print("\n✓ Setup complete! Ready to proceed.")

In [ ]:
"""
CELL 2: Load and Preprocess MNIST Dataset
"""

print("=" * 70)
print("DATA LOADING AND PREPROCESSING")
print("=" * 70)

# Load the MNIST dataset
print("\n[Step 1] Loading MNIST Dataset...")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(f"✓ Training samples: {x_train.shape[0]}")
print(f"✓ Test samples: {x_test.shape[0]}")
print(f"✓ Image shape: {x_train.shape[1:]}")

# Preprocess the data
print("\n[Step 2] Preprocessing data...")

# Normalize pixel values to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Reshape to add channel dimension (samples, height, width, channels)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print(f"✓ Training data shape: {x_train.shape}")
print(f"✓ Test data shape: {x_test.shape}")

# Convert labels to categorical
num_classes = 10
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

print(f"✓ Number of classes: {num_classes}")
print(f"✓ Label encoding: Categorical (one-hot)")

# Visualize sample images
print("\n[Step 3] Visualizing sample training images...")
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('Sample Training Images from MNIST', fontsize=14, fontweight='bold')

for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i].squeeze(), cmap='gray')
    ax.set_title(f'Label: {y_train[i]}', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n✓ Data preprocessing complete!")
print("=" * 70)

In [ ]:
"""
CELL 3: Build  CNN Architecture 

"""

print("=" * 70)
print("IMPROVED CNN MODEL V1 ARCHITECTURE")
print("=" * 70)

# Build the Improved CNN model V1
print("\n[Building Model] Creating Improved Convolutional Neural Network...")

model = keras.Sequential([
    # ========== CONVOLUTIONAL BLOCK 1 ==========
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", 
                  input_shape=(28, 28, 1), padding='same', name="conv1_1"),
    layers.BatchNormalization(name="bn1_1"),
    
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", 
                  padding='same', name="conv1_2"),
    layers.BatchNormalization(name="bn1_2"),
    
    layers.MaxPooling2D(pool_size=(2, 2), name="pool1"),
    layers.Dropout(0.25, name="dropout1"),
    
    # ========== CONVOLUTIONAL BLOCK 2 ==========
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", 
                  padding='same', name="conv2_1"),
    layers.BatchNormalization(name="bn2_1"),
    
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", 
                  padding='same', name="conv2_2"),
    layers.BatchNormalization(name="bn2_2"),
    
    layers.MaxPooling2D(pool_size=(2, 2), name="pool2"),
    layers.Dropout(0.25, name="dropout2"),
    
    # ========== CONVOLUTIONAL BLOCK 3 (NEW!) ==========
    layers.Conv2D(128, kernel_size=(3, 3), activation="relu", 
                  padding='same', name="conv3_1"),
    layers.BatchNormalization(name="bn3_1"),
    
    layers.Conv2D(128, kernel_size=(3, 3), activation="relu", 
                  padding='same', name="conv3_2"),
    layers.BatchNormalization(name="bn3_2"),
    
    layers.Dropout(0.25, name="dropout3"),
    
    # ========== DENSE LAYERS ==========
    layers.Flatten(name="flatten"),
    
    layers.Dense(256, activation="relu", name="dense1"),
    layers.BatchNormalization(name="bn_dense1"),
    layers.Dropout(0.5, name="dropout_dense1"),
    
    layers.Dense(128, activation="relu", name="dense2"),
    layers.BatchNormalization(name="bn_dense2"),
    layers.Dropout(0.5, name="dropout_dense2"),
    
    layers.Dense(num_classes, activation="softmax", name="output")
], name="Improved_MNIST_CNN_V1")

# Display model architecture
print("\n[Model Summary]")
model.summary()

# Compile the model
print("\n[Compiling Model]")
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print("\n✓ Improved Model V1 architecture created successfully!")
print(f"✓ Total parameters: {model.count_params():,}")
print("\n🎯 Key Improvements:")
print("  • Added 3rd convolutional block (128 filters)")
print("  • Batch Normalization after each layer")
print("  • Two Conv layers per block for deeper feature extraction")
print("  • Increased dense layer capacity (256 → 128)")
print("=" * 70)

In [ ]:
"""
CELL 4: Train the CNN Model with Callbacks
"""

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print("=" * 70)
print("MODEL TRAINING WITH CALLBACKS")
print("=" * 70)

batch_size = 128
epochs = 25

print(f"\n[Training Configuration]")
print(f"Batch size: {batch_size}")
print(f"Max Epochs: {epochs}")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Loss function: Categorical Crossentropy")
print(f"Validation split: 10%")

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=7,
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("\n[Callbacks Enabled]")
print("  - Early Stopping (patience=7)")
print("  - Learning Rate Reduction (patience=3)")
print("  - Model Checkpoint (save best)")

print("\n[Starting Training]")
print("-" * 70)

history = model.fit(
    x_train, y_train_cat,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "=" * 70)
print("MODEL TRAINING COMPLETED")
print("=" * 70)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
actual_epochs = len(history.history['accuracy'])

print(f"\nTraining Summary:")
print(f"  Actual Epochs Trained: {actual_epochs} (out of {epochs} max)")
print(f"  Final Training Accuracy: {final_train_acc * 100:.2f}%")
print(f"  Final Validation Accuracy: {final_val_acc * 100:.2f}%")
print(f"  Final Training Loss: {final_train_loss:.4f}")
print(f"  Final Validation Loss: {final_val_loss:.4f}")

overfitting_gap = (final_train_acc - final_val_acc) * 100
if overfitting_gap > 2:
    print(f"\n  Warning: Possible overfitting detected")
    print(f"  Training-Validation gap: {overfitting_gap:.2f}%")
else:
    print(f"\n  Good generalization - Train-Val gap: {overfitting_gap:.2f}%")

print("=" * 70)

In [ ]:
"""
CELL 5: Visualize Training History
"""

print("=" * 70)
print("TRAINING HISTORY VISUALIZATION")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
axes[0].plot(history.history['accuracy'], label='Training Accuracy', 
             linewidth=2, marker='o')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', 
             linewidth=2, marker='s')
axes[0].set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Plot loss
axes[1].plot(history.history['loss'], label='Training Loss', 
             linewidth=2, marker='o')
axes[1].plot(history.history['val_loss'], label='Validation Loss', 
             linewidth=2, marker='s')
axes[1].set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
save_path = os.path.join(OUTPUT_DIR, 'training_history_baseline.png')
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✓ Training history saved as '{save_path}'")
plt.show()

print("\n✓ Training visualization complete!")
print("=" * 70)

In [ ]:
"""
CELL 6: Evaluate Baseline Model Performance
Activity 2: Baseline Performance Metrics
"""

print("=" * 70)
print("BASELINE MODEL PERFORMANCE EVALUATION")
print("=" * 70)

# 1. ACCURACY AND LOSS
print("\n[Metric 1] Computing Accuracy and Loss on Clean Test Set...")
test_loss, test_accuracy = model.evaluate(x_test, y_test_cat, verbose=0)

print(f"\n{'=' * 50}")
print(f"BASELINE PERFORMANCE METRICS")
print(f"{'=' * 50}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"{'=' * 50}")

# Generate predictions for further analysis
print("\n[Generating Predictions]")
y_pred = model.predict(x_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
print(f"✓ Predictions generated for {len(y_test)} test samples")

print("\n✓ Basic evaluation complete!")
print("=" * 70)

In [ ]:
"""
CELL 7: Confusion Matrix Analysis
"""

print("=" * 70)
print("CONFUSION MATRIX ANALYSIS")
print("=" * 70)

# Generate confusion matrix
print("\n[Metric 2] Generating Confusion Matrix...")
cm = confusion_matrix(y_test, y_pred_classes)

# Display confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True, 
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Baseline Model', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
save_path = os.path.join(OUTPUT_DIR, 'confusion_matrix_baseline.png')
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✓ Confusion matrix saved as '{save_path}'")
plt.show()

# Calculate per-class accuracy
print("\n[Per-Class Performance]")
print("-" * 50)
print(f"{'Digit':<10} {'Accuracy':<15} {'Misclassified':<15}")
print("-" * 50)

per_class_accuracies = []
for i in range(num_classes):
    class_total = cm[i, :].sum()
    class_correct = cm[i, i]
    class_accuracy = (class_correct / class_total) * 100
    class_errors = class_total - class_correct
    per_class_accuracies.append(class_accuracy)
    print(f"{i:<10} {class_accuracy:>6.2f}%{'':<8} {class_errors:>6}")

print("-" * 50)
avg_per_class_acc = np.mean(per_class_accuracies)
print(f"Average per-class accuracy: {avg_per_class_acc:.2f}%")

print("\n✓ Confusion matrix analysis complete!")
print("=" * 70)

In [ ]:
"""
CELL 8: Detailed Classification Report
"""

print("=" * 70)
print("CLASSIFICATION REPORT")
print("=" * 70)

# Generate classification report
print("\n[Metric 3] Detailed Per-Class Metrics:")
print("-" * 70)
report = classification_report(y_test, y_pred_classes, 
                               target_names=[f"Digit {i}" for i in range(10)],
                               digits=4)
print(report)

# Get classification report as dictionary for further analysis
report_dict = classification_report(y_test, y_pred_classes, 
                                    target_names=[f"Digit {i}" for i in range(10)],
                                    output_dict=True)

# Create a DataFrame for better visualization
metrics_df = pd.DataFrame(report_dict).transpose()
metrics_df = metrics_df.round(4)

print("\n[Summary Statistics]")
print(f"Overall Accuracy: {report_dict['accuracy']:.4f}")
print(f"Macro Avg Precision: {report_dict['macro avg']['precision']:.4f}")
print(f"Macro Avg Recall: {report_dict['macro avg']['recall']:.4f}")
print(f"Macro Avg F1-Score: {report_dict['macro avg']['f1-score']:.4f}")

print("\n✓ Classification report complete!")
print("=" * 70)

In [ ]:
"""
CELL 9: Inference Time Measurement
"""

print("=" * 70)
print("INFERENCE TIME MEASUREMENT")
print("=" * 70)

# Measure inference time for different batch sizes
print("\n[Metric 4] Measuring Inference Time...")

# Single sample inference
print("\n[Test 1] Single Sample Inference:")
single_sample = x_test[0:1]
start_time = time.time()
_ = model.predict(single_sample, verbose=0)
end_time = time.time()
single_time = (end_time - start_time) * 1000
print(f"  Time per sample: {single_time:.2f} ms")

# Batch inference (100 samples)
print("\n[Test 2] Batch Inference (100 samples):")
num_samples = 100
batch_data = x_test[:num_samples]
start_time = time.time()
_ = model.predict(batch_data, verbose=0)
end_time = time.time()
batch_time = end_time - start_time
avg_time_per_sample = (batch_time / num_samples) * 1000
throughput = num_samples / batch_time

print(f"  Total time: {batch_time:.4f} seconds")
print(f"  Average time per sample: {avg_time_per_sample:.2f} ms")
print(f"  Throughput: {throughput:.2f} samples/second")

# Large batch inference (1000 samples)
print("\n[Test 3] Large Batch Inference (1000 samples):")
num_samples_large = 1000
large_batch_data = x_test[:num_samples_large]
start_time = time.time()
_ = model.predict(large_batch_data, verbose=0)
end_time = time.time()
large_batch_time = end_time - start_time
avg_time_large = (large_batch_time / num_samples_large) * 1000
throughput_large = num_samples_large / large_batch_time

print(f"  Total time: {large_batch_time:.4f} seconds")
print(f"  Average time per sample: {avg_time_large:.2f} ms")
print(f"  Throughput: {throughput_large:.2f} samples/second")

# Summary
print("\n" + "=" * 70)
print("INFERENCE PERFORMANCE SUMMARY")
print("=" * 70)
print(f"Single sample latency: {single_time:.2f} ms")
print(f"Batch (100) avg latency: {avg_time_per_sample:.2f} ms/sample")
print(f"Batch (1000) avg latency: {avg_time_large:.2f} ms/sample")
print(f"Maximum throughput: {throughput_large:.2f} samples/second")
print("=" * 70)

print("\n✓ Inference time measurement complete!")

In [ ]:
"""
CELL 10: Sample Predictions Visualization
"""

print("=" * 70)
print("SAMPLE PREDICTIONS VISUALIZATION")
print("=" * 70)

# Visualize correct and incorrect predictions
print("\n[Visualization] Displaying sample predictions...")

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Predictions on Test Set', fontsize=16, fontweight='bold')

for i, ax in enumerate(axes.flat):
    ax.imshow(x_test[i].squeeze(), cmap='gray')
    pred_label = y_pred_classes[i]
    true_label = y_test[i]
    confidence = y_pred[i][pred_label] * 100
    
    # Color code: green for correct, red for incorrect
    color = 'green' if pred_label == true_label else 'red'
    ax.set_title(f'Pred: {pred_label} ({confidence:.1f}%)\nTrue: {true_label}', 
                 color=color, fontsize=10, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
save_path = os.path.join(OUTPUT_DIR, 'sample_predictions_baseline.png')
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✓ Sample predictions saved as '{save_path}'")
plt.show()

# Find and display misclassified examples
print("\n[Analysis] Finding Misclassified Examples...")
misclassified_idx = np.where(y_pred_classes != y_test)[0]
print(f"Total misclassified samples: {len(misclassified_idx)} out of {len(y_test)}")
print(f"Error rate: {(len(misclassified_idx)/len(y_test)*100):.2f}%")

if len(misclassified_idx) > 0:
    print("\n[Displaying First 10 Misclassifications]")
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle('Misclassified Examples', fontsize=16, fontweight='bold', color='red')
    
    for i, ax in enumerate(axes.flat):
        if i < len(misclassified_idx):
            idx = misclassified_idx[i]
            ax.imshow(x_test[idx].squeeze(), cmap='gray')
            pred_label = y_pred_classes[idx]
            true_label = y_test[idx]
            confidence = y_pred[idx][pred_label] * 100
            ax.set_title(f'Pred: {pred_label} ({confidence:.1f}%)\nTrue: {true_label}', 
                        color='red', fontsize=10, fontweight='bold')
        ax.axis('off')
    
    plt.tight_layout()
    save_path = os.path.join(OUTPUT_DIR, 'misclassified_examples.png')
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"✓ Misclassified examples saved as '{save_path}'")
    plt.show()

print("\n✓ Prediction visualization complete!")
print("=" * 70)

In [ ]:
"""
CELL 12: Final Summary and Output Files
"""

print("=" * 70)
print("PHASE 1 COMPLETE - FINAL SUMMARY")
print("=" * 70)

print(f"\n📁 Output Directory: {OUTPUT_DIR}")
print("\n📊 Generated Files:")
print("-" * 70)

# List all generated files with details
file_list = []
for filename in sorted(os.listdir(OUTPUT_DIR)):
    filepath = os.path.join(OUTPUT_DIR, filename)
    size = os.path.getsize(filepath) / 1024  # Convert to KB
    file_list.append({
        'Filename': filename,
        'Size (KB)': round(size, 2),
        'Type': os.path.splitext(filename)[1][1:].upper()
    })

df_files = pd.DataFrame(file_list)
print(df_files.to_string(index=False))
print("-" * 70)

# Key Metrics Summary
print("\n📈 Key Performance Metrics:")
print("-" * 70)
print(f"✓ Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"✓ Test Loss: {test_loss:.4f}")
print(f"✓ Inference Time: {avg_time_per_sample:.2f} ms/sample")
print(f"✓ Throughput: {throughput_large:.2f} samples/second")
print(f"✓ Model Parameters: {model.count_params():,}")
print(f"✓ Misclassification Rate: {(len(misclassified_idx)/len(y_test)*100):.2f}%")
print("-" * 70)

# Files description
print("\n📝 File Descriptions:")
print("-" * 70)
descriptions = {
    'confusion_matrix_baseline.png': 'Visual confusion matrix showing classification patterns',
    'training_history_baseline.png': 'Training/validation accuracy and loss curves',
    'sample_predictions_baseline.png': 'Sample test predictions with confidence scores',
    'misclassified_examples.png': 'Examples of misclassified digits for error analysis',
    'baseline_performance.csv': 'Complete performance metrics in tabular format',
    'mnist_cnn_baseline.h5': 'Trained model in HDF5 format (TensorFlow 1.x compatible)',
    'mnist_cnn_baseline.keras': 'Trained model in Keras format (TensorFlow 2.x recommended)',
    'model_architecture.json': 'Model architecture definition in JSON format'
}

for filename, description in descriptions.items():
    if filename in [f['Filename'] for f in file_list]:
        print(f"  • {filename}")
        print(f"    {description}")
        print()

print("=" * 70)
print("PHASE 1: MODEL CREATION AND BASELINE PERFORMANCE - COMPLETED")
print("=" * 70)